## Customer Life Time Value (CLTV) Calculation on Retail Dataset

##### Link of data: https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

##### In this project I tried to apply Customer Life Time Value calculation and Customer segmentation methods to retail dataset above

### Imports

In [1]:
import datetime as dt
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

##### This is how you read data from your own device
##### There are multiple sheets which are consist of different years
##### In this project we will focus on 2010-2011 sheet

In [2]:
df_ = pd.read_excel("retail.xlsx", sheet_name="Year 2010-2011")

In [45]:
df = df_.copy() # You better take a copy of your data. Because the data is hard to load in case of screw up :)

### Data Preperation

In [ ]:
df.head()

In [13]:
df.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [16]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925.0,13.021793,180.419984,1.0,1.00,1.00,1.00,2.00,6.00,12.00,24.00,36.0,120.00,80995.00
Price,397925.0,3.116212,22.096773,0.0,0.21,0.42,0.55,1.25,1.95,3.75,6.35,8.5,14.95,8142.75
Customer ID,397925.0,15294.308601,1713.172738,12346.0,12415.00,12627.00,12883.00,13969.00,15159.00,16795.00,17725.00,17912.0,18211.00,18287.00


##### In this data set as can be seen Customer ID and Invoice variables are consist of dublicated records
##### This means a customer can have more than one invoice and an invoice can have more than one items

##### There are some Null values lets get rid of them

In [3]:
df.dropna(axis=0, inplace=True)


##### As mentioned in data source there are some cancelled Invoices in the data set.
##### Cancelled invoices are represented with a capital "C" in the Invoice variable.
##### We need to get rid of them too.

In [4]:
df = df[~df["Invoice"].str.contains("C", na=False)]

##### If we look at varience of Quantity and Price variables we can see that the max values are rediculusly high.
##### Lets pressure them down with the following functions

In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    return up_limit


def replace_with_thresholds(dataframe, variable):
    up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

##### Lets check again

In [6]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925.0,11.833709,25.534486,1.0,1.00,1.00,1.00,2.00,6.00,12.00,24.00,36.0,120.00,298.50
Price,397925.0,2.893201,3.227143,0.0,0.21,0.42,0.55,1.25,1.95,3.75,6.35,8.5,14.95,37.06
Customer ID,397925.0,15294.308601,1713.172738,12346.0,12415.00,12627.00,12883.00,13969.00,15159.00,16795.00,17725.00,17912.0,18211.00,18287.00


##### Looks great
##### Lets create a variable called "Total Price" by multiplying Price and Quantity variables to calculate the amount paid in total

In [7]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

### Calculating Customer Life Time Values (CLTV)

##### To start we need to have recency, frequency, monetary variables
##### recency: means how many days have passed since customers last purches
##### frequency: means how many purches a customer had until today
##### monetary: means how much money spent by a customer umtil today
##### T: means how many days have passed since customers first purches (Customer's Age)
##### Lets create them

##### But what is today? We need to define a "today" first :)

In [8]:
df["InvoiceDate"].max()

today_date = dt.datetime(2011, 12, 11)

##### We need to take a look at max value in "InvoiceDate" variable and chose a date later than the maximum value 
##### so that we dont get negative values in recency variable we will create in a bit
##### thats why we chose df["InvoiceDate"].max() + 2 days

##### Now we can create rfm table

In [20]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                    lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

##### Lets see our rfm table loks like

In [12]:
rfm.head()

,recency_cltv_p,T,frequency,monetary
Customer ID,,,,
12346.0,0,326,1,310.44
12347.0,365,368,7,4310.00
12348.0,282,359,4,1770.78
12349.0,0,19,1,1491.72
12350.0,0,311,1,331.46


In [13]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
recency_cltv_p,4339.0,130.418530,132.039186,0.0,0.0000,0.000,0.00,0.000,92.00,251.500,339.000,356.000,367.0000,373.000
T,4339.0,223.782899,117.883623,1.0,10.0000,26.000,46.00,113.000,249.00,327.000,367.000,372.000,374.0000,374.000
frequency,4339.0,4.271952,7.705493,1.0,1.0000,1.000,1.00,1.000,2.00,5.000,9.000,13.000,30.0000,210.000
monetary,4339.0,1891.748117,7705.372078,0.0,50.8419,110.908,154.87,303.125,662.59,1630.445,3579.054,5633.161,16944.7554,266163.525


##### As can be seen there are customers with "0" monetary which is weird because they got invoices but the amount is "0"
##### Lets get rid of them
##### Also to calculate a life time value we better select customers which has more than one purcheses
##### Lets do that too 

In [21]:
rfm = rfm[rfm["monetary"] > 0]

rfm = rfm[(rfm['frequency'] > 1)]

##### Now lets change "monetary" variable into "avg_monetary" by diveding "frequency" variable to calculate avarage monetary value per purches

In [22]:
rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)

##### Lets check again if it all worked out

In [23]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg
Customer ID,,,,
12347.0,365,368,7,615.714286
12348.0,282,359,4,442.695000
12352.0,260,297,8,219.542500
12356.0,302,326,3,937.143333
12358.0,149,151,2,575.210000


In [18]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
recency_cltv_p,2845.0,198.905448,113.866961,0.00,0.0000,16.000000,35.000,98.00,207.000000,299.00,352.000000,361.800000,369.0000,373.000000
T,2845.0,258.784183,102.990922,3.00,23.4400,55.000000,85.000,192.00,283.000000,359.00,369.000000,373.000000,374.0000,374.000000
frequency,2845.0,5.990158,9.054720,2.00,2.0000,2.000000,2.000,2.00,4.000000,6.00,11.600000,17.000000,37.0000,210.000000
monetary_avg,2845.0,372.140959,349.147893,3.45,52.0572,101.978667,132.256,191.07,300.487308,436.87,646.278067,867.951667,1732.1836,5724.302619


##### Everything looks allright

##### To calculate cltv we will use BGNBD Model and Gama Gama Model. In order to use this model we neet to change our "Recency" and "T"(Customer Age) variables into weeks

In [25]:
rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
rfm["T_weekly"] = rfm["T"] / 7

##### Now we are good to go for building BGNBD Model

In [26]:
bgf = BetaGeoFitter(penalizer_coef=0.01)
bgf.fit(rfm['frequency'],
        rfm['recency_weekly_cltv_p'],
        rfm['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.06, alpha: 9.30, b: 1.13, r: 1.78>

##### With this model we are able to calculate expected purchases of every single customer in the data set. 
##### Also if we sum up all these purchases we can get the expected revenue of the company

In [30]:
# exp_sales_1_month
rfm["exp_sales_1_month"] = bgf.predict(4, # <= The time period can be changed here (remember that our time period is in weeks)
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'])
# exp_sales_3_month
rfm["exp_sales_3_month"] = bgf.predict(12, # <= The time period can be changed here
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'])
#Note: Here we are not calculating how much money will be spent. We are calculating how many purcheses one customer is likely to make

##### Now we will use Gama Gama model to calculate the expected average profit per customer

In [31]:
# expected_average_profit
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(rfm['frequency'], rfm['monetary_avg'])
rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                         rfm['monetary_avg'])

##### By combining bgf(BGNBD Model) and ggf(Gama Gama Model) we can calculate CLTV for different time periods

In [32]:
cltv = ggf.customer_lifetime_value(bgf,
                                   rfm['frequency'],
                                   rfm['recency_weekly_cltv_p'],
                                   rfm['T_weekly'],
                                   rfm['monetary_avg'],
                                   time=6,   # Here is where we decleare the time period(months) that CLTV will be calculated
                                   freq="W", # Here is where we inform our model that in our data the time period is in weeks(w)
                                   discount_rate=0.01) 

rfm["cltv_p"] = cltv

##### Lets check again if it all worked out

In [38]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p
Customer ID,,,,,,,,,,
12347.0,365,368,7,615.714286,52.142857,52.571429,0.561684,1.678069,631.911974,2.933517
12348.0,282,359,4,442.695000,40.285714,51.285714,0.364322,1.087951,463.745539,1.919391
12352.0,260,297,8,219.542500,37.142857,42.428571,0.739160,2.206857,224.886669,1.904146
12356.0,302,326,3,937.143333,43.142857,46.571429,0.333040,0.993934,995.997679,2.802492
12358.0,149,151,2,575.210000,21.285714,21.571429,0.474048,1.408955,631.900951,2.612984


##### Everything looks allright

##### Now in order "cltv_p" score we calculated to make sense to us lets make it scaleable

In [37]:
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(rfm[["cltv_p"]])
rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

##### We can now confidently segment our customers according to their cltv values

In [39]:
rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 4, labels=["bronze","silver", "gold", "premium"])

##### Here we diveded customers into segments using quartiles but this segmentation can be customized with using more complex structures

##### Now We are succesfully segmented our customers. Lets sort them down and take a look

In [44]:
rfm.sort_values(by="cltv_p", ascending=False).head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,
14646.0,353,355,74,3596.804392,50.428571,50.714286,5.035170,15.049808,3605.309159,100.000000,premium
18102.0,366,368,60,3859.739083,52.285714,52.571429,3.981335,11.900989,3870.996702,85.065982,premium
14096.0,97,102,17,3163.588235,13.857143,14.571429,3.112278,9.255671,3196.435385,54.640819,premium
12415.0,313,338,21,5724.302619,44.714286,48.285714,1.560391,4.662433,5772.177190,50.081947,premium
14911.0,372,374,201,691.710100,53.142857,53.428571,12.896742,38.554879,692.326355,49.715635,premium
